<a href="https://colab.research.google.com/github/Looma1116/Kaggle-Study_Seungjoon_Lee/blob/main/w9_CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DataSet= UniversalBank : Loan or Not

In [92]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

bank_df = pd.read_csv('/content/drive/MyDrive/DataFolder/UniversalBank.csv')
bank_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [93]:
x = bank_df.drop(['ID','ZIP Code','Personal Loan'], axis=1)
y = bank_df['Personal Loan']

In [94]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.4, stratify=y)

##Model Setting  
DecisionTree, KNN, LogisticRegression

In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(solver='liblinear',
                              penalty ='l2',
                              C=0.01,
                              random_state=1)
tree = DecisionTreeClassifier(max_depth=None,
                              criterion='entropy',
                              random_state=1)
knn =KNeighborsClassifier(n_neighbors=4,
                          p=2,
                          metric='minkowski')

#Ensemble : Majority Voting

In [96]:
from sklearn.ensemble import VotingClassifier
voting_estimators = [('logistic', logistic), ('tree', tree), ('knn',knn)]
voting = VotingClassifier(estimators = voting_estimators, voting='soft')

K-fold 교차검증

In [97]:
from sklearn.model_selection import cross_val_score
clf_labels = ['Logistic regression', 'Decision tree', 'KNN', 'Majority voting']
all_clf = [logistic, tree, knn, voting]

for clf, label in zip(all_clf, clf_labels):                     #cv(closs validation) : k-fold 몇 겹)
   scores = cross_val_score(estimator=clf, X = x_train, y=y_train, cv=10, scoring = 'roc_auc')
   print("ROC AUC: %0.3f (+/- %0.3f) [%s]" % (scores.mean(),scores.std(),label))

ROC AUC: 0.940 (+/- 0.021) [Logistic regression]
ROC AUC: 0.950 (+/- 0.031) [Decision tree]
ROC AUC: 0.850 (+/- 0.040) [KNN]
ROC AUC: 0.978 (+/- 0.017) [Majority voting]


##GridSearch 
 방식을 통해 각각의 모델의 최적 파라피터를 찾아보자

In [98]:
from sklearn.model_selection import GridSearchCV

params = {'logistic__C' : [0.001, 10, 100],
          'tree__max_depth' : [1, 3, 5],
          'knn__n_neighbors' : [1, 3, 5]
}

grid = GridSearchCV(estimator=voting,
                    param_grid=params,
                    cv=10,
                    scoring = 'roc_auc',)
grid.fit(x_train,y_train)

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
  print('%0.3f +/- %0.3f %r' % (grid.cv_results_['mean_test_score'][r],
                                grid.cv_results_['std_test_score'][r] / 2.0,
                                grid.cv_results_['params'][r]))

print('최적의 파라미터: %s' % grid.best_params_)
print('ACU: %.3f' % grid.best_score_)


0.924 +/- 0.009 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 1}
0.970 +/- 0.006 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 3}
0.973 +/- 0.006 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 5}
0.942 +/- 0.008 {'knn__n_neighbors': 1, 'logistic__C': 10, 'tree__max_depth': 1}
0.978 +/- 0.006 {'knn__n_neighbors': 1, 'logistic__C': 10, 'tree__max_depth': 3}
0.978 +/- 0.007 {'knn__n_neighbors': 1, 'logistic__C': 10, 'tree__max_depth': 5}
0.942 +/- 0.008 {'knn__n_neighbors': 1, 'logistic__C': 100, 'tree__max_depth': 1}
0.978 +/- 0.006 {'knn__n_neighbors': 1, 'logistic__C': 100, 'tree__max_depth': 3}
0.978 +/- 0.007 {'knn__n_neighbors': 1, 'logistic__C': 100, 'tree__max_depth': 5}
0.928 +/- 0.010 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 1}
0.976 +/- 0.005 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 3}
0.979 +/- 0.006 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 5}
0.950 +

# Ensemble : Bagging

In [99]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(base_estimator=tree,
                            n_estimators=500, #트리가 500개!?
                            max_samples=1.0,
                            max_features=1.0,
                            bootstrap=True, #입력데이터 중복 허용
                            bootstrap_features=False,
                            n_jobs=1,
                            random_state=1)

K-fold

In [100]:
clf_labels = ['Decision tree', 'Bagging']
all_clf = [tree, bagging]

for clf, label in zip(all_clf, clf_labels):
  scores = cross_val_score(estimator= clf, X=x_train, y = y_train, cv=10, scoring='roc_auc')
  print("ROC AUC: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))

ROC AUC: 0.950 (+/- 0.031) [Decision tree]
ROC AUC: 0.998 (+/- 0.001) [Bagging]


# Ensemble : Boosting

In [101]:
tree = DecisionTreeClassifier(max_depth=1,
                              criterion = 'entropy',
                              random_state=1)

from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(base_estimator=tree,
                              n_estimators=500,
                              learning_rate=0.1,
                              random_state=1)


In [102]:
clf_labels = ['Decision tree', 'Boosting']
all_clf = [tree, adaboost]

for clf, label in zip(all_clf, clf_labels):
  scores = cross_val_score(estimator= clf, X=x_train, y = y_train, cv=10, scoring='roc_auc')
  print("ROC AUC: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))

ROC AUC: 0.863 (+/- 0.021) [Decision tree]
ROC AUC: 0.980 (+/- 0.012) [Boosting]


#Pipe Line

In [112]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline


pipe = make_pipeline(StandardScaler(), PCA(n_components=10), DecisionTreeClassifier())

param_range1 = [1, 3, 5, 7]
param_range2 = [10, 20, 30, 40, 50]

param_grid = [{'decisiontreeclassifier__max_depth' : param_range1,
               'decisiontreeclassifier__min_samples_leaf' : param_range2}]

GS = GridSearchCV(estimator=pipe,
                  param_grid = param_grid,
                  scoring = 'accuracy',
                  cv = 5,
                  n_jobs = -1 #using all processors 
                  )
GS = GS.fit(x_train, y_train)

print(GS.best_score_)
print(GS.best_params_)

0.9373333333333334
{'decisiontreeclassifier__max_depth': 7, 'decisiontreeclassifier__min_samples_leaf': 10}
